In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import random
import os
import pathlib

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip

In [ ]:
zip_ref = zipfile.ZipFile("10_food_classes_all_data.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [ ]:
!ls 10_food_classes_all_data

In [ ]:
for dirpath, dirnames, filenames in os.walk("10_food_classes_all_data"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")


In [ ]:
class_names = [item.name for item in pathlib.Path('10_food_classes_all_data/train/').glob('*')]
class_names = np.array(sorted(class_names))
class_names

In [ ]:
class_names1 = np.array(sorted(os.listdir('10_food_classes_all_data/train')))
class_names == class_names1

In [ ]:
img_path = '10_food_classes_all_data/train/hamburger/'+(random.sample(os.listdir('10_food_classes_all_data/train/hamburger'),1))[0]
img = plt.imread(img_path)
plt.imshow(img)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_path = "10_food_classes_all_data/train"
test_data_path = "10_food_classes_all_data/test"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    directory = train_data_path,
    target_size=(224, 224),
    color_mode='rgb', 
    classes=None, 
    class_mode='categorical', 
    batch_size=32, 
    shuffle=True, 
    seed=42,
)

test_data = test_datagen.flow_from_directory(
    directory=test_data_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

In [ ]:
model_0 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(10, 3, activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.Conv2D(10, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(10, 3, activation='relu'),
    tf.keras.layers.Conv2D(10, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

model_0.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

history_0 = model_0.fit(
    train_data,
    steps_per_epoch=len(train_data),
    epochs=10,
    validation_data=test_data,
    validation_steps=len(test_data)
)

model_hist_df = pd.DataFrame(history_0.history)
model_hist_df.plot()
plt.xlabel('epochs')

In [ ]:
model_0.evaluate(test_data)

In [ ]:
probs = model_0.predict(test_data) 
probs[:10]

In [ ]:
preds = probs.argmax(axis=1)
class_names[preds[:10]]

In [ ]:
model_0.save("saved_multiclass_cnn_model")

In [ ]:
loaded_model = tf.keras.models.load_model("saved_multiclass_cnn_model")
loaded_model.evaluate(test_data)